In [1]:
import io
import os
import sys
import csv
import gzip
import time
import json
import secrets
import multiprocessing
import subprocess
import resource
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import statsmodels.api as sm
import random
from collections import Counter
import seaborn as sns
import matplotlib.colors as mcolors
import itertools
import collections
sys.path.append('/well/band/users/rbx225/software/lcwgsus/')
# sys.path.append('/Users/sus_zhang/Desktop/Suuuuuuuus/lcwgsus/')
import lcwgsus

from scipy.stats import poisson
from scipy.stats import chi2
from scipy.stats import friedmanchisquare
from scipy.stats import studentized_range
pd.options.mode.chained_assignment = None
%cd /gpfs3/well/band/users/rbx225/GAMCC/

# from matplotlib import colormaps
# list(colormaps)

/gpfs3/well/band/users/rbx225/GAMCC


In [2]:
chip_vcf = '/well/band/users/rbx225/GAMCC/results/lr_imputation/truth/lr.vcf.gz'
lc_vcf = '/well/band/users/rbx225/GAMCC/results/lr_imputation/vcfs/oneKG/quilt.vcf.gz'
af_txt = "data/gnomAD_MAFs/afr/gnomAD_MAF_afr_chr21.txt"

chip, lc, af = lcwgsus.imputation_calculation_preprocess(chip_vcf, lc_vcf, af_txt)

In [10]:
def calculate_v_imputation_accuracy(chip,
                                    lc,
                                    af,
                                    MAF_ary=lcwgsus.MAF_ARY,
                                    impacc_colnames=lcwgsus.IMPACC_V_COLS,
                                    common_cols=lcwgsus.COMMON_COLS,
                                    chip_sample_prefix=lcwgsus.CHIP_SAMPLE_PREFIX,
                                    lc_sample_prefix=lcwgsus.LC_SAMPLE_PREFIX,
                                    save_file=False,
                                    outdir=None,
                                    save_name=None):
    stack_lst = [[] for _ in range(len(impacc_colnames))]

    chip_af = pd.merge(chip, af, on=common_cols)
    lc_af = pd.merge(lc, af, on=common_cols)

    chip_samples = lcwgsus.valid_sample(chip.iloc[0, :])
    lc_samples = lcwgsus.valid_sample(lc.iloc[0, :])
    n_sample = len(chip_samples)

    for chip_sample, lc_sample in zip(chip_samples, lc_samples):
        chip = chip_af[['MAF', chip_sample]]
        lc = lc_af[['MAF', lc_sample]]

        for i in range(np.size(MAF_ary)):
            if i == 0:
                tmp_chip = chip[chip_af['MAF'] == 0]
                tmp_lc = lc_af[lc_af['MAF'] == 0]
            else:
                tmp_chip = chip[(MAF_ary[i] > chip['MAF'])
                                & (chip['MAF'] > MAF_ary[i - 1])]
                tmp_lc = lc[(MAF_ary[i] > lc['MAF'])
                            & (lc['MAF'] > MAF_ary[i - 1])]

            r1 = np.array(tmp_chip[chip_sample].values).astype(float)
            r2 = np.array(tmp_lc[lc_sample].values).astype(float)
            tmp_metrics = lcwgsus.calculate_imputation_accuracy_metrics(r1, r2)
            metrics = [chip_sample, MAF_ary[i], tmp_chip.shape[0]]
            metrics = lcwgsus.fix_v_metrics(metrics, tmp_metrics)
            stack_lst = lcwgsus.append_lst(metrics, stack_lst)

    res_df = pd.DataFrame(dict(zip(impacc_colnames, stack_lst)))
    return res_df

In [14]:
v_report = lcwgsus.calculate_h_imputation_accuracy(chip, lc, af)
v_report

,chr,pos,ref,alt,MAF,NRC,NRC_BC,r2,r2_BC,ccd_homref,ccd_homref_BC,ccd_het,ccd_het_BC,ccd_homalt,ccd_homalt_BC
0,21,5030578,C,T,0.041403,-9.0,0,-9,5,1.0,5,-9.0,0,-9.0,0
1,21,5030588,T,C,0.023582,-9.0,0,-9,5,1.0,5,-9.0,0,-9.0,0
2,21,5030596,A,G,0.000173,-9.0,0,-9,5,1.0,5,-9.0,0,-9.0,0
3,21,5030673,G,A,0.000067,-9.0,0,-9,5,1.0,5,-9.0,0,-9.0,0
4,21,5030957,G,A,0.083738,-9.0,0,-9,5,1.0,5,-9.0,0,-9.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6543,21,7266387,A,C,0.000199,-9.0,0,-9,5,1.0,5,-9.0,0,-9.0,0
6544,21,7266411,A,G,0.000099,-9.0,0,-9,5,1.0,5,-9.0,0,-9.0,0
6545,21,7266444,A,G,0.002669,-9.0,0,-9,5,1.0,5,-9.0,0,-9.0,0
6546,21,7266463,C,T,0.001287,-9.0,0,-9,5,1.0,5,-9.0,0,-9.0,0


In [12]:
lcwgsus.generate_v_impacc(v_report)

,AF,n_variants,NRC,NRC_BC,NRC_AC,r2,r2_BC,r2_AC,ccd_homref,ccd_homref_BC,ccd_homref_AC,ccd_het,ccd_het_BC,ccd_het_AC,ccd_homalt,ccd_homalt_BC,ccd_homalt_AC
0,0.0000,0,-9.000000,0.0,0,-9.000000,0.0,0,-9.000000,0.0,0,-9.000000,0.0,0,-9.000000,0.0,0
1,0.0001,1200,0.000000,1.8,9,0.006134,1200.0,6000,0.999333,1199.0,5995,0.000000,1.0,5,-9.000000,0.0,0
2,0.0002,532,0.000000,1.2,6,0.112514,532.0,2660,0.999623,531.0,2655,0.000000,1.0,5,-9.000000,0.0,0
3,0.0005,786,0.055556,3.6,18,0.045841,786.0,3930,0.997959,784.0,3920,0.100000,2.0,10,-9.000000,0.0,0
4,0.0010,578,0.000000,2.2,11,0.004639,578.0,2890,0.997920,577.0,2885,0.000000,1.0,5,-9.000000,0.0,0
5,0.0020,655,0.000000,5.2,26,0.024210,655.0,3275,0.998157,651.0,3255,0.000000,4.0,20,-9.000000,0.0,0
6,0.0050,884,0.036145,16.6,83,0.066567,884.0,4420,0.997011,870.0,4350,0.042857,14.0,70,-9.000000,0.0,0
7,0.0100,591,0.051282,7.8,39,0.048816,591.0,2955,0.995222,586.0,2930,0.080000,5.0,25,-9.000000,0.0,0
8,0.0200,388,0.058824,10.2,51,0.053635,388.0,1940,0.991601,381.0,1905,0.085714,7.0,35,-9.000000,0.0,0
9,0.0500,358,0.201493,26.8,134,0.195049,358.0,1790,0.991617,334.0,1670,0.225000,24.0,120,-9.000000,0.0,0


In [7]:
calculate_v_imputation_accuracy

NameError: name 'calculate_v_imputation_accuracy' is not defined